# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 18 2022 8:30AM,248798,10,8659875,Bio-PRF,Released
1,Oct 18 2022 8:30AM,248800,10,0086081414,ISDIN Corporation,Released
2,Oct 18 2022 8:30AM,248800,10,0086081489,ISDIN Corporation,Released
3,Oct 18 2022 8:30AM,248800,10,0086081496,ISDIN Corporation,Released
4,Oct 18 2022 8:30AM,248800,10,0086081492,ISDIN Corporation,Released
5,Oct 18 2022 8:30AM,248801,10,8659886,Yusen – 3D Matrix,Released
6,Oct 18 2022 8:30AM,248803,10,MSP214950,"Methapharm, Inc.",Released
7,Oct 18 2022 8:30AM,248803,10,MSP214953,"Methapharm, Inc.",Released
8,Oct 18 2022 8:30AM,248803,10,MSP214955,"Methapharm, Inc.",Released
9,Oct 18 2022 8:30AM,248803,10,MSP214960,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,248803,Released,4
32,248804,Released,1
33,248805,Released,9
34,248806,Released,1
35,248807,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248803,NaN,NaN,4.0
248804,NaN,NaN,1.0
248805,NaN,NaN,9.0
248806,NaN,NaN,1.0
248807,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248635,0.0,0.0,1.0
248636,0.0,0.0,1.0
248707,53.0,2.0,0.0
248719,9.0,3.0,2.0
248734,7.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248635,0,0,1
248636,0,0,1
248707,53,2,0
248719,9,3,2
248734,7,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248635,0,0,1
1,248636,0,0,1
2,248707,53,2,0
3,248719,9,3,2
4,248734,7,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248635,,,1
1,248636,,,1
2,248707,53,2,
3,248719,9,3,2
4,248734,7,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation
5,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix
6,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc."
10,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc."
19,Oct 17 2022 3:59PM,248807,19,ACG North America LLC
20,Oct 17 2022 3:52PM,248806,19,ACG North America LLC
21,Oct 17 2022 3:46PM,248804,19,ACG North America LLC
22,Oct 17 2022 3:30PM,248796,20,"ACI Healthcare USA, Inc."
23,Oct 17 2022 3:19PM,248795,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF,,,1
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation,,,4
2,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix,,,1
3,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc.",,,4
4,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc.",,,9
5,Oct 17 2022 3:59PM,248807,19,ACG North America LLC,,,1
6,Oct 17 2022 3:52PM,248806,19,ACG North America LLC,,,1
7,Oct 17 2022 3:46PM,248804,19,ACG North America LLC,,,1
8,Oct 17 2022 3:30PM,248796,20,"ACI Healthcare USA, Inc.",,,1
9,Oct 17 2022 3:19PM,248795,10,ISDIN Corporation,,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF,1,,
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation,4,,
2,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix,1,,
3,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc.",4,,
4,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc.",9,,
5,Oct 17 2022 3:59PM,248807,19,ACG North America LLC,1,,
6,Oct 17 2022 3:52PM,248806,19,ACG North America LLC,1,,
7,Oct 17 2022 3:46PM,248804,19,ACG North America LLC,1,,
8,Oct 17 2022 3:30PM,248796,20,"ACI Healthcare USA, Inc.",1,,
9,Oct 17 2022 3:19PM,248795,10,ISDIN Corporation,6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF,1,,
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation,4,,
2,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix,1,,
3,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc.",4,,
4,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc.",9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF,1.0,NaN,NaN
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation,4.0,NaN,NaN
2,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc.",4.0,NaN,NaN
4,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc.",9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 8:30AM,248798,10,Bio-PRF,1.0,0.0,0.0
1,Oct 18 2022 8:30AM,248800,10,ISDIN Corporation,4.0,0.0,0.0
2,Oct 18 2022 8:30AM,248801,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Oct 18 2022 8:30AM,248803,10,"Methapharm, Inc.",4.0,0.0,0.0
4,Oct 18 2022 8:30AM,248805,10,"Methapharm, Inc.",9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2985459,53.0,9.0,0.0
12,248707,0.0,2.0,53.0
15,497453,4.0,3.0,16.0
18,248780,1.0,0.0,0.0
19,1492706,22.0,11.0,1.0
20,746346,4.0,0.0,0.0
21,497271,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2985459,53.0,9.0,0.0
1,12,248707,0.0,2.0,53.0
2,15,497453,4.0,3.0,16.0
3,18,248780,1.0,0.0,0.0
4,19,1492706,22.0,11.0,1.0
5,20,746346,4.0,0.0,0.0
6,21,497271,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,53.0,9.0,0.0
1,12,0.0,2.0,53.0
2,15,4.0,3.0,16.0
3,18,1.0,0.0,0.0
4,19,22.0,11.0,1.0
5,20,4.0,0.0,0.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,12,Released,0.0
2,15,Released,4.0
3,18,Released,1.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21
Status,,,,,,,
Completed,0.0,53.0,16.0,0.0,1.0,0.0,0.0
Executing,9.0,2.0,3.0,0.0,11.0,0.0,0.0
Released,53.0,0.0,4.0,1.0,22.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21
0,Completed,0.0,53.0,16.0,0.0,1.0,0.0,0.0
1,Executing,9.0,2.0,3.0,0.0,11.0,0.0,0.0
2,Released,53.0,0.0,4.0,1.0,22.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21
0,Completed,0.0,53.0,16.0,0.0,1.0,0.0,0.0
1,Executing,9.0,2.0,3.0,0.0,11.0,0.0,0.0
2,Released,53.0,0.0,4.0,1.0,22.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()